# Passo a Passo - Básico

In [ ]:
# ler arquivo
import pandas as pd

file_path = "..."
data = pd.read_csv(file_path)

In [ ]:
# escolher o "alvo" (prediction target)
y = data.nome_coluna_alvo

In [ ]:
# escolher as "features", as características
data_features = ['coluna_x1', 'coluna_x2', 'coluna_x3']

X = data[data_features]

In [ ]:
# dividir os dados para treino e validação
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y  = train_test_split(random_state = 0)

In [ ]:
# verificar se existem valores faltantes

## existem 3 maiores abordagens:
## - excluir linhas com dados faltantes
## - imputação - atribuir aos valores faltantes a média dos valores de cada coluna
## - extensão da imputação - faz a imputação padrão e também uma nova coluna para mostrar onde a imputação foi feita





In [ ]:
# construindo modelo
# 1) definir tipo de modelo (ex. árvore de decisão)
from sklearn.tree import DecisionTreeRegressor
data_model = DecisionTreeRegressor(random_state = 1)

In [ ]:
# 2) ajustar modelo as variáveis 
data_model.fit(train_X, train_y)

In [ ]:
# 3) fazer previsões
val_predictions = data_model.predict(val_X)

In [ ]:
# 4) validar o modelo

## pode ser validado pelo erro médio absoluto (Mean Absolute Error - MAE)

from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(val_y, val_predictions))

In [ ]:
# comparando os resultados obtidos com os resultados atuais/reais
print('Previsão: ', val_predictions[:5])
print('Atual: ', val_y.head().tolist())